<a href="https://colab.research.google.com/github/johnfriesen/Population_Model/blob/main/Global_population_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required libraries
import glob
import os
import ee
import geopandas as gpd
import pandas as pd

# Authenticate with Google Earth Engine
# (For first-time users, you might need to run ee.Authenticate() separately)
ee.Authenticate()
project_id =
ee.Initialize(project=project_id)

# List all shapefiles that match the pattern <country>_segments.shp
shapefile_paths = glob.glob("*_segments.shp")

# Access the ImageCollection from Earth Engine
col = ee.ImageCollection('GOOGLE/Research/open-buildings-temporal/v1')

# Function to sum the building_fractional_count for a specific geometry at a given timestamp
def sum_building_fractional_count(geometry, millis):
    # Create a mosaic of tiles with the same timestamp
    mosaic = col.filter(ee.Filter.eq('system:time_start', millis)).mosaic()
    fractional_count_map = mosaic.select('building_fractional_count')
    total_sum = fractional_count_map.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=5,  # Adjust the scale as needed
        maxPixels=1e9
    )
    return total_sum.get('building_fractional_count').getInfo()

# List to hold results for all countries
all_results = []

# Loop through each shapefile (each country's data)
for shapefile_path in shapefile_paths:
    # Extract country name from filename (e.g., "nigeria_segments.shp" -> "nigeria")
    country = os.path.basename(shapefile_path).split('_segments')[0]
    print(f"\nProcessing country: {country}")

    # Load the shapefile using geopandas
    gdf = gpd.read_file(shapefile_path)

    # Get the list of unique cities from the 'UC_NM_LST' column
    unique_cities = gdf['UC_NM_LST'].unique()

    # Process each city in the shapefile
    for city in unique_cities:
        # Filter the GeoDataFrame for rows where 'UC_NM_LST' matches the city and 'SES_2CAT_U' equals 1
        filtered_gdf = gdf[(gdf['UC_NM_LST'] == city) & (gdf['SES_2CAT_U'] == 1)]

        if filtered_gdf.empty:
            print(f"  No entries found for {city} in {country} with SES_2CAT_U equal to 1")
            continue

        # Merge geometries for the city
        filtered_geometry = filtered_gdf.geometry.unary_union

        # Convert the merged geometry to GeoJSON and then to an Earth Engine Geometry
        filtered_geojson = gpd.GeoSeries([filtered_geometry]).__geo_interface__['features'][0]['geometry']
        filtered_ee_geometry = ee.Geometry(filtered_geojson)

        # Retrieve available timestamps and select the latest one
        ts_list = col.filterBounds(filtered_ee_geometry) \
                     .aggregate_array('system:time_start') \
                     .distinct() \
                     .sort() \
                     .getInfo()

        if not ts_list:
            print(f"  No timestamp found for {city} in {country}")
            continue

        latest_ts = ts_list[-1]

        try:
            # Compute the total building count (multiplied by 100 as in your original code)
            total_sum_value = sum_building_fractional_count(filtered_ee_geometry, latest_ts) * 100

            # Estimate populations using different factors
            estimated_population_5 = total_sum_value * 5
            estimated_population_3_5 = total_sum_value * 3.5
            estimated_population_6_5 = total_sum_value * 6.5

            # Append the result for this city
            all_results.append({
                'Country': country,
                'City': city,
                'Total Buildings': total_sum_value,
                'Estimated Population (5)': estimated_population_5,
                'Estimated Population (3.5)': estimated_population_3_5,
                'Estimated Population (6.5)': estimated_population_6_5
            })

            print(f"  Processed {city}: Total Buildings = {total_sum_value}, Populations = {estimated_population_5}, {estimated_population_3_5}, {estimated_population_6_5}")

        except Exception as e:
            print(f"  Error processing {city} in {country}: {e}")

# Save all the results to a CSV file
results_df = pd.DataFrame(all_results)
results_df.to_csv("all_countries_building_population_estimates.csv", index=False)
print("\nProcessing complete. Results saved to 'all_countries_building_population_estimates.csv'.")


In [ ]:
# Import required libraries
import glob
import os
import ee
import geopandas as gpd
import pandas as pd

# Authenticate with Google Earth Engine
# For first-time users, you might need to run ee.Authenticate() separately.
ee.Authenticate()
project_id =
ee.Initialize(project=project_id)

# Mapping from country name in the filename to the ISO code used in WorldPop
# Update this dictionary with additional mappings as needed.
country_iso_mapping = {
    "nigeria": "NGA",
    "kenya": "KEN",
    "tanzania": "TZA",
    "zimbabwe":"ZWE",
    "zambia":"ZWB",
    "yemen":"YEM",
    "western_sahara":"ESH",
    "vietnam":"VNM",
    "rwanda":"RWA",
    "venezuela":"VEN",
    "uzbekistan":"UZB",
    "thailand":"THA",
    "sri_lanka":"LKA",
    "uganda":"UGA",
    "peru":"PER",
    "pakistan":"PAK",
    "namibia":"NAM",
    "mexico":"MEX",
    "mali":"MLI",
    "mongolia":"MNG",
    "nepal":"NPL",
    "niger":"NER"

    # Add more mappings here if you have more countries
}

# List all shapefiles matching the pattern <country>_segments.shp
shapefile_paths = glob.glob("*_segments.shp")

# Specify the year for which you want to retrieve population estimates
year = 2020  # Change as desired

# Function to sum the WorldPop population for a specific geometry and country
def sum_worldpop_population(geometry, year, iso_code):
    """
    Sums up the values in the WorldPop population band for the given region
    and country ISO code.
    """
    # Filter the WorldPop collection for the specific country and year
    worldpop_col = ee.ImageCollection("WorldPop/GP/100m/pop") \
                     .filter(ee.Filter.eq('country', iso_code)) \
                     .filter(ee.Filter.eq('year', year))

    population_image = worldpop_col.first()
    if population_image is None:
        raise ValueError(f"No population data available for {iso_code} in {year}")

    # Compute the sum of population values over the region
    total_population = population_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=100,  # WorldPop data has 100m resolution
        maxPixels=1e9
    )

    return total_population.get('population').getInfo()

# List to store results for all countries and cities
pop_results = []

# Loop through each shapefile (each representing one country)
for shapefile_path in shapefile_paths:
    # Extract the country name from the filename (e.g., "nigeria_segments.shp" -> "nigeria")
    country_name = os.path.basename(shapefile_path).split('_segments')[0].lower()

    # Get the corresponding ISO code from the mapping dictionary
    iso_code = country_iso_mapping.get(country_name)
    if iso_code is None:
        print(f"ISO code for {country_name} not found in mapping. Skipping...")
        continue

    print(f"\nProcessing country: {country_name} (ISO: {iso_code})")

    # Load the shapefile using geopandas
    gdf = gpd.read_file(shapefile_path)

    # Get the list of unique cities from the 'UC_NM_LST' column
    unique_cities = gdf['UC_NM_LST'].unique()

    # Process each city in the shapefile
    for city in unique_cities:
        # Filter the GeoDataFrame for rows where 'UC_NM_LST' matches the city and 'SES_2CAT_U' equals 1
        filtered_gdf = gdf[(gdf['UC_NM_LST'] == city) & (gdf['SES_2CAT_U'] == 1)]

        if filtered_gdf.empty:
            print(f"  No entries found for {city} in {country_name} with SES_2CAT_U equal to 1")
            continue

        # Merge all geometries for the city into a single geometry
        filtered_geometry = filtered_gdf.geometry.unary_union

        # Convert the merged geometry to GeoJSON and then to an Earth Engine Geometry
        filtered_geojson = gpd.GeoSeries([filtered_geometry]).__geo_interface__['features'][0]['geometry']
        filtered_ee_geometry = ee.Geometry(filtered_geojson)

        try:
            # Calculate the total population for this city
            total_population_value = sum_worldpop_population(filtered_ee_geometry, year, iso_code)

            # Append the results for this city
            pop_results.append({
                'Country': country_name,
                'ISO Code': iso_code,
                'City': city,
                'Year': year,
                'Estimated Population': total_population_value
            })

            print(f"  Processed {city}: Estimated Population in {year} = {total_population_value}")
        except Exception as e:
            print(f"  Error processing {city} in {country_name}: {e}")

# Save the results to a CSV file
pop_results_df = pd.DataFrame(pop_results)
pop_results_df.to_csv("all_countries_worldpop_population_estimates.csv", index=False)
print("\nProcessing complete. Results saved to 'all_countries_worldpop_population_estimates.csv'.")


Create a Worldmap showing the differences between Worldpop and the building population model.

In [ ]:
pip install cartopy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

# ---------------------------
# 1. Load CSV files from multi-country analysis
# ---------------------------
# Building-based estimates (from your multi-country code)
building_population_df = pd.read_csv("all_countries_building_population_estimates.csv")
# WorldPop estimates (from your multi-country code)
worldpop_population_df = pd.read_csv("all_countries_worldpop_population_estimates.csv")

# ---------------------------
# 2. Merge DataFrames on Country and City
# ---------------------------
merged_df = pd.merge(building_population_df, worldpop_population_df, on=["Country", "City"], how="inner",
                     suffixes=("_Building", "_WorldPop"))


print(merged_df.head())
# ---------------------------
# 3. Compute per-city statistics and percentage differences
# ---------------------------
# Calculate per-city mean and standard deviation for building-based estimates (across factors 3.5, 5, and 6.5)
merged_df['Building_Estimate_Mean'] = merged_df[['Estimated Population (3.5)',
                                                  'Estimated Population (5)',
                                                  'Estimated Population (6.5)']].mean(axis=1)
merged_df['Building_Estimate_Std'] = merged_df[['Estimated Population (3.5)',
                                                 'Estimated Population (5)',
                                                 'Estimated Population (6.5)']].std(axis=1)

# Compute the percent difference per city (using the factor 5 estimate as representative)
merged_df['Percent_Difference'] = ((merged_df['Estimated Population (5)'] - merged_df['Estimated Population']) /
                                   merged_df['Estimated Population']) * 100

# Aggregate the percent differences by country (average per country)
country_diff = merged_df.groupby('Country', as_index=False)['Percent_Difference'].mean()
country_diff['Country'] = country_diff['Country'].str.title()  # standardize country names

# Use Cartopy's shapereader to obtain admin-0 countries (resolution 110m)
shpfilename = shpreader.natural_earth(resolution='110m', category='cultural', name='admin_0_countries')
world = gpd.read_file(shpfilename)

# Standardize the country names for merging (the shapefile uses a "NAME" column)
world['Country'] = world['NAME'].str.title()

# Merge our computed country differences with the world boundaries
merged_map = world.merge(country_diff, how='left', on='Country')

# ---------------------------
# 5. Plot the choropleth map using Cartopy
# ---------------------------
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE)
ax.set_title("Mean Percentage Difference by Country\n(Building-Based (Factor 5) vs. WorldPop)", fontsize=14)

# Plot the merged map with percentage differences
merged_map.plot(column='Percent_Difference',
                cmap='coolwarm',
                legend=True,
                ax=ax,
                transform=ccrs.PlateCarree(),
                missing_kwds={'color': 'lightgrey',
                              'edgecolor': 'orange',
                              'hatch': '///',
                              'label': 'No data'})
plt.show()



# ---------------------------
# 6. Create a basic (i.e., "bad") bar plot comparing country differences
# ---------------------------
plt.figure(figsize=(10, 6))
plt.bar(country_diff['Country'], country_diff['Percent_Difference'], color='skyblue')
plt.xlabel('Country')
plt.ylabel('Mean Percent Difference (%)')
plt.title('Country-by-Country Mean Percent Difference\n(Building-Based (Factor 5) vs. WorldPop)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# --- Assume merged_df is already created from merging your CSV files ---
# For example:
# merged_df = pd.merge(building_population_df, worldpop_population_df, on=["Country", "City"], how="inner",
#                      suffixes=("_Building", "_WorldPop"))
# And then compute:
# merged_df['Building_Estimate_Mean'] = merged_df[['Estimated Population (3.5)', 'Estimated Population (5)', 'Estimated Population (6.5)']].mean(axis=1)
# merged_df['Building_Estimate_Std'] = merged_df[['Estimated Population (3.5)', 'Estimated Population (5)', 'Estimated Population (6.5)']].std(axis=1)

# Create a color mapping for each country using a colormap
countries = merged_df['Country'].unique()
cmap = plt.cm.get_cmap('tab10', len(countries))
color_dict = {country: cmap(i) for i, country in enumerate(countries)}

plt.figure(figsize=(10, 8))

# Plot each city as a point with an error bar (y-axis error: building std)
for idx, row in merged_df.iterrows():
    plt.errorbar(x=row['Estimated Population'],
                 y=row['Building_Estimate_Mean'],
                 yerr=row['Building_Estimate_Std'],
                 fmt='o',
                 color=color_dict[row['Country']],
                 capsize=3,
                 alpha=0.8)

# Draw the identity line where Building-Based Estimate equals WorldPop Estimate
# Calculate the limits of the plot based on the data
x_min = min(merged_df['Estimated Population'].min(), merged_df['Building_Estimate_Mean'].min())
x_max = max(merged_df['Estimated Population'].max(), merged_df['Building_Estimate_Mean'].max())
plt.plot([x_min, x_max], [x_min, x_max], 'k--', label='y = x (Equality)')

# Create dummy scatter points to build a legend for the countries
for country in countries:
    plt.scatter([], [], color=color_dict[country], label=country)

plt.xlabel('WorldPop Population Estimate')
#plt.xscale('log')
#plt.yscale('log')
plt.ylabel('Building-Based Population Estimate (Mean ± STD)')
plt.title('City-Level Population Estimates: WorldPop vs. Building-Based')
plt.legend(title='Country', loc='best')
plt.tight_layout()
plt.show()


Calculate national population estimates based on building model and worldpop.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------
# 1. Load CSV files
# ---------------------------
df_building = pd.read_csv("all_countries_building_population_estimates.csv")
df_worldpop = pd.read_csv("all_countries_worldpop_population_estimates.csv")

# ---------------------------
# 2. Group and Sum Data by Country
# ---------------------------
# Sum each of the three factors for building-based estimates per country
group_building = df_building.groupby("Country").agg({
    "Estimated Population (3.5)": "sum",
    "Estimated Population (5)": "sum",
    "Estimated Population (6.5)": "sum"
})
# Sum WorldPop estimates per country
group_worldpop = df_worldpop.groupby("Country")["Estimated Population"].sum()

# ---------------------------
# 3. Convert Totals to Millions
# ---------------------------
group_building = group_building / 1e6
group_worldpop = group_worldpop / 1e6

# For building-based estimates, use factor 5 as the representative value
total_building_mean = group_building["Estimated Population (5)"]
# Use the factor 3.5 sum as the lower bound and factor 6.5 sum as the upper bound
total_building_lower = group_building["Estimated Population (3.5)"]
total_building_upper = group_building["Estimated Population (6.5)"]

# ---------------------------
# 4. Compute Asymmetric Error Bars
# ---------------------------
# Error bars represent the difference from the mean to the lower and upper bounds
error_lower = total_building_mean - total_building_lower  # how much above the lower bound
error_upper = total_building_upper - total_building_mean  # how much below the upper bound

# Combine errors into a 2-row array for asymmetric error bars
errors = np.vstack((error_lower.values, error_upper.values))

# ---------------------------
# 5. Combine Data into a Single DataFrame for Plotting
# ---------------------------
plot_df = pd.DataFrame({
    "Country": total_building_mean.index,
    "Building": total_building_mean.values,
    "WorldPop": group_worldpop.reindex(total_building_mean.index).values,
    "Error_Lower": error_lower.values,
    "Error_Upper": error_upper.values
})

print("Population Totals (in millions):")
print(plot_df)

# ---------------------------
# 6. Plot the Grouped Bar Chart with Error Bars
# ---------------------------
countries = plot_df["Country"]
x = np.arange(len(countries))
width = 0.35  # width of each bar

fig, ax = plt.subplots(figsize=(10, 6))

# Plot building-based totals with error bars (asymmetric)
rects1 = ax.bar(x - width/2, plot_df["Building"], width,
                yerr=errors, capsize=5, label="Building-Based", color="skyblue")

# Plot WorldPop totals (no error bars)
rects2 = ax.bar(x + width/2, plot_df["WorldPop"], width,
                label="WorldPop", color="orange")

ax.set_ylabel("Total Population (Millions)")
ax.set_title("Total Population Estimates (in Millions) by Country")
ax.set_xticks(x)
ax.set_xticklabels(countries, rotation=45, ha="right")
ax.legend()

# Optionally, add text labels on top of the bars
def autolabel(rects, values):
    for rect, value in zip(rects, values):
        height = rect.get_height()
        ax.annotate(f'{value:.1f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # vertical offset
                    textcoords="offset points",
                    ha="center", va="bottom")

autolabel(rects1, plot_df["Building"])
autolabel(rects2, plot_df["WorldPop"])

plt.tight_layout()
plt.show()


Calculate global population estimates based on building model and worldpop.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------
# 1. Load CSV files
# ---------------------------
df_building = pd.read_csv("all_countries_building_population_estimates.csv")
df_worldpop = pd.read_csv("all_countries_worldpop_population_estimates.csv")

# ---------------------------
# 2. Compute Overall Totals (summing over all rows, assuming each row is a unique city)
# ---------------------------
overall_building_3_5 = df_building["Estimated Population (3.5)"].sum()
overall_building_5   = df_building["Estimated Population (5)"].sum()
overall_building_6_5 = df_building["Estimated Population (6.5)"].sum()
overall_worldpop     = df_worldpop["Estimated Population"].sum()

# ---------------------------
# 3. Convert Totals to Millions
# ---------------------------
overall_building_3_5 /= 1e6
overall_building_5   /= 1e6
overall_building_6_5 /= 1e6
overall_worldpop     /= 1e6

# ---------------------------
# 4. Compute Asymmetric Error Bars for Building-Based Estimates
# ---------------------------
# Use the factor 5 total as the representative value.
# Lower error: difference between factor 5 and factor 3.5 totals.
# Upper error: difference between factor 6.5 and factor 5 totals.
error_lower = overall_building_5 - overall_building_3_5
error_upper = overall_building_6_5 - overall_building_5
# Create an array with the two error values (shape: 2 x 1) for asymmetric error bars.
building_errors = np.array([[error_lower], [error_upper]])

# ---------------------------
# 5. Plot Overall Totals Comparison
# ---------------------------
fig, ax = plt.subplots(figsize=(6, 6))

# Plot the Building-Based overall total with asymmetric error bars.
ax.bar("Building-Based", overall_building_5, yerr=building_errors, capsize=5,
       color="skyblue", label="Building-Based")
# Plot the WorldPop overall total (no uncertainty shown).
ax.bar("WorldPop", overall_worldpop, color="orange", label="WorldPop")

ax.set_ylabel("Total Population (Millions)")
ax.set_title("Overall Total Population Estimates (in Millions)")
ax.legend()

plt.tight_layout()
plt.show()
